# LittleMCMC Quickstart

LittleMCMC is a lightweight and performant implementation of HMC and NUTS in Python, spun out of the PyMC project. In this quickstart tutorial, we will introduce LittleMCMC

## Table of Contents

- [Who should use LittleMCMC?](#Who-should-use-LittleMCMC?)
- [Sampling](#Sampling)
  - [Inspecting the Output of `lmc.sample`](#Inspecting-the-Output-of-lmc.sample)
- [Other Modules](#Other-Modules)

## Who should use LittleMCMC?

<div class="alert alert-block alert-info">
LittleMCMC is a fairly bare bones library with a very niche use case. Most users will probably find that [PyMC3](https://github.com/pymc-devs/pymc3) will satisfy their needs, with better strength of support and quality of documentation.
</div>

If you:

1. Have model with only continuous parameters,
1. Are willing to manually "unconstrain" all of your model's parameters (if necessary),
1. Have methods to compute the log probability of the model and its derivative, exposed via a Python callable,
1. And all you need is an implementation of HMC/NUTS (preferably in Python) to sample from your model

then you should consider using LittleMCMC!

## Sampling

In [1]:
import numpy as np
import scipy
import littlemcmc as lmc

In [2]:
def logp_func(x, loc=0, scale=1):
    return np.log(scipy.stats.norm.pdf(x, loc=loc, scale=scale))


def dlogp_func(x, loc=0, scale=1):
    return -(x - loc) / scale


def logp_dlogp_func(x, loc=0, scale=1):
    return logp_func(x, loc=loc, scale=scale), dlogp_func(x, loc=loc, scale=scale)

In [3]:
trace, stats, results = lmc.sample(
    logp_dlogp_func=logp_dlogp_func,
    size=1,
    draws=1000,
    tune=500,
    step=lmc.NUTS(logp_dlogp_func=logp_dlogp_func, size=1),
    chains=4,
    cores=4,
    progressbar="notebook"
)

/Users/george/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/Users/george/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


### Inspecting the Output of `lmc.sample`

In [4]:
trace

array([-0.38331274, -1.76994233, -0.67234733, ...,  0.27817656,
        0.29250676,  0.42966184])

In [5]:
trace.shape

(4000,)

In [6]:
stats

{'depth': array([1, 1, 1, ..., 1, 2, 1]),
 'step_size': array([0.94586326, 0.94586326, 0.94586326, ..., 2.16938615, 2.16938615,
        2.16938615]),
 'tune': array([False, False, False, ..., False, False, False]),
 'mean_tree_accept': array([1.        , 0.43665689, 1.        , ..., 0.98765583, 0.72296808,
        0.97965297]),
 'step_size_bar': array([1.20597596, 1.20597596, 1.20597596, ..., 1.28614833, 1.28614833,
        1.28614833]),
 'tree_size': array([1., 1., 1., ..., 1., 3., 1.]),
 'diverging': array([False, False, False, ..., False, False, False]),
 'energy_error': array([-0.25675836,  0.82860753, -0.74393026, ...,  0.01242099,
         0.00169732,  0.02055688]),
 'energy': array([1.25393394, 2.56056236, 1.91071276, ..., 0.95981431, 1.76229677,
        1.02575724]),
 'max_energy_error': array([-0.25675836,  0.82860753, -0.74393026, ...,  0.01242099,
         0.56981615,  0.02055688]),
 'model_logp': array([-0.99240286, -2.48528646, -1.144964  , ..., -0.95762963,
        -0.961

In [7]:
stats["diverging"].shape

(4000,)

## Other Modules

LittleMCMC exposes:

1. Two step methods: Hamiltonian Monte Carlo (HMC) and the No-U-Turn Sampler (NUTS)
1. Quadpotentials (a.k.a. mass matrices or inverse metrics)
1. Dual-averaging step size adaptation
1. Leapfrog integration

Refer to the [API Reference](https://littlemcmc.readthedocs.io/en/latest/api.html) for more information.